In [1]:
import os
import time
import pandas as pd
import numpy as np

from astropy.io import ascii
from astropy.coordinates import SkyCoord
from SPLASH.pipeline import Splash_Pipeline

# Load in data

In [2]:
# Load in 100 bts host grizy observations
try:
    bts_hosts = ascii.read('demo_bts_hosts.ecsv')
except FileNotFoundError:
    os.chdir('SPLASH_demo')
    bts_hosts = ascii.read('demo_bts_hosts.ecsv')
bts_hosts = bts_hosts[1:]  # drop annoying row
bts_df = bts_hosts.to_pandas()
bts_df.head()

,objID_3pi,raStack_3pi,decStack_3pi,primaryDetection_3pi,gKronMag_3pi,rKronMag_3pi,iKronMag_3pi,zKronMag_3pi,yKronMag_3pi,gKronMagErr_3pi,rKronMagErr_3pi,iKronMagErr_3pi,zKronMagErr_3pi,yKronMagErr_3pi,ps_score_3pi,SN_ra,SN_dec,sn_class,sn_redshift,ZTFID
0,148231465149294617,146.514940,33.528361,1,18.2387,17.9746,17.8750,17.8532,17.7821,0.004157,0.004871,0.004281,0.014407,0.019814,0.021125,146.514792,33.528250,SN II,0.03800,ZTF18aacemcn
1,153731349546771375,134.954717,38.108970,1,18.7478,19.0585,18.6466,18.8301,18.6221,0.006392,0.009475,0.004919,0.017235,0.038667,0.017411,134.954667,38.109056,SN II,0.07247,ZTF18aacnlxz
2,169101206490177628,120.649031,50.922461,1,16.5325,15.6522,15.2551,15.0064,14.7582,0.002335,0.002128,0.000867,0.001623,0.003512,0.075905,120.648958,50.922528,SN Ia,0.05295,ZTF18aadlaxo
3,159661026075384557,102.607536,43.053266,1,NaN,14.2309,13.1660,12.9816,12.8047,NaN,0.000836,0.000408,0.001003,0.001687,0.154649,102.607542,43.053222,SN IIn,0.01885,ZTF18aadmssd
4,146001295340652041,129.534082,31.667894,1,17.2694,16.9697,16.8672,16.8232,16.7636,0.004272,0.003413,0.005182,0.005542,0.014564,0.004667,129.533958,31.667917,SN II,0.03057,ZTF18aadsuxd


In [3]:
def get_angular_separation(ra1, dec1, ra2, dec2, unit1='deg', unit2='deg'):
    """Function to calculate angular separation in arcseconds"""
    coord1 = SkyCoord(ra1, dec1, unit=unit1)
    coord2 = SkyCoord(ra2, dec2, unit=unit2)
    return coord1.separation(coord2).arcsec

def ab_mag_to_flux(AB_mag: np.ndarray) -> np.ndarray:
    """Convert AB magnitude to flux in mJy"""
    return 10**((AB_mag - 8.9) / -2.5) * 1000

ab_magerr_to_ferr = lambda sigma_m, f: np.abs(f * np.log(10) * (sigma_m / 2.5))  # transformation on the error of a magnitude turned into flux

# Add angular separation and grib data
bts_df['angular_separation_arcsec'] = get_angular_separation(bts_df['raStack_3pi'], bts_df['decStack_3pi'], bts_df['SN_ra'], bts_df['SN_dec'], unit1='deg', unit2='deg')
grizy = bts_df[['gKronMag_3pi', 'rKronMag_3pi', 'iKronMag_3pi', 'zKronMag_3pi', 'yKronMag_3pi']].to_numpy().astype(float)
grizy_err = bts_df[['gKronMagErr_3pi', 'rKronMagErr_3pi', 'iKronMagErr_3pi', 'zKronMagErr_3pi', 'yKronMagErr_3pi']].to_numpy().astype(float)
angular_seps = bts_df['angular_separation_arcsec'].to_numpy().astype(float)

# Convert the grizy data to mJy
grizy = ab_mag_to_flux(grizy)
grizy_err = ab_magerr_to_ferr(grizy_err, grizy)

grizy, angular_seps

(array([[1.83873863e-01, 2.34509262e-01, 2.57039578e-01, 2.62252719e-01,
         2.80001269e-01],
        [1.15048245e-01, 8.64171621e-02, 1.26287394e-01, 1.06649789e-01,
         1.29169506e-01],
        [8.85115610e-01, 1.99122346e+00, 2.87051619e+00, 3.60944144e+00,
         4.53649044e+00],
        [           nan, 7.37292812e+00, 1.96607463e+01, 2.33002188e+01,
         2.74233180e+01],
        [4.48993444e-01, 5.91725111e-01, 6.50309353e-01, 6.77204757e-01,
         7.15418226e-01],
        [2.98483274e-01, 6.53792599e-01, 1.09415790e+00, 1.42902557e+00,
         1.69855651e+00],
        [6.01893981e-01, 1.23720025e+00, 1.92415476e+00, 2.25029782e+00,
         2.59680895e+00],
        [1.73988282e+00,            nan,            nan,            nan,
                    nan],
        [3.22760218e+00, 5.96815371e+00, 8.96107269e+00, 1.00110585e+01,
         1.09496440e+01],
        [4.09449175e-01, 8.94046279e-01, 1.34091114e+00, 1.71316818e+00,
         2.08276370e+00],
        [2

In [4]:
nan_mask = ~(np.sum(np.isnan(grizy), axis=1)==0)

# Use the SPLASH pipeline

In [5]:
# Load pipeline object
pipeline = Splash_Pipeline(
    pre_transformed=False,      # whether the given data is pre-logged and nnormalized
    within_4sigma=True,         # whether we only want to classify objects with properties within 4-sigma of the training set
    nan_thresh_ratio=1.0,       # to keep this notebook concise, we are allowing the pipeline to imput any num of nans
)

In [11]:
# Predict the classes. n_resamples is the number of boostraps for getting the median predicted host properties.
start_time = time.time()
classes_only_phot = pipeline.infer_classes(grizy=grizy, redshift=np.array(bts_hosts['sn_redshift'].filled(np.nan)), angular_sep=angular_seps, grizy_err=grizy_err, n_resamples=50)
duration = time.time() - start_time
print(f'{grizy.shape[0]} classifications produced in {duration} seconds (~{duration / grizy.shape[0]} seconds per classificaiton).')

99 classifications produced in 0.06890988349914551 seconds (~0.0006960594292842981 seconds per classificaiton).


In [15]:
print(f'Number of each class:\n{pd.Series(classes_only_phot).value_counts()}\n\nThe class labels are\n0=Ia\n1=Ib/c\n2=SLSN\n3=IIn\n4=II (P/L)\n-1=Outside train properties 4 sigma\n-2=No host found')

Number of each class:
0    92
4     7
Name: count, dtype: int64

The class labels are
0=Ia
1=Ib/c
2=SLSN
3=IIn
4=II (P/L)
-1=Outside train properties 4 sigma
-2=No host found


In [13]:
# We can also take a look at the predicted host properties of each supernova
props, props_err = pipeline.infer_host_properties(grizy=grizy, redshift=np.array(bts_hosts['sn_redshift'].filled(np.nan)), grizy_err=grizy_err, n_resamples=50, return_normalized=False)  # return_normalized will return the properties normalized with the train mean and std
props, props_err  # in order (log(mass), log(sfr), redshift)

(array([[ 8.96030474e+00, -1.09284876e+00],
        [ 8.86660804e+00, -5.37448039e-01],
        [ 1.07131061e+01, -1.27656469e+00],
        [ 1.05151734e+01,  2.99547517e-01],
        [ 9.19497656e+00, -8.82698422e-01],
        [ 1.03862756e+01, -1.03665597e+00],
        [ 1.03944637e+01, -9.68966905e-01],
        [ 1.05663692e+01, -5.56251249e-02],
        [ 1.02992324e+01, -2.18445250e-01],
        [ 1.04928076e+01, -1.15004314e+00],
        [ 1.08764599e+01, -1.08227383e-01],
        [ 1.02998103e+01, -9.10195161e-01],
        [ 9.78862361e+00, -3.51083124e-01],
        [ 1.09631863e+01, -1.22674035e-01],
        [ 1.04394108e+01, -9.93962244e-01],
        [ 9.86848474e+00, -2.79566896e-01],
        [ 1.03347036e+01,  7.55876043e-02],
        [ 9.76961174e+00, -8.67952162e-01],
        [ 1.02123146e+01,  2.83352396e-02],
        [ 1.04002912e+01, -2.00662565e-01],
        [ 8.73715062e+00, -1.29268491e+00],
        [ 1.04775170e+01, -4.65414969e-01],
        [ 1.08330441e+01, -7.903

# Classification from just transient coordinates

In [21]:
# Load pipeline object
pipeline = Splash_Pipeline(
    pre_transformed=False,      # whether the given data is pre-logged and nnormalized
    within_4sigma=True,         # whether we only want to classify objects with properties within 4-sigma of the training set
    nan_thresh_ratio=1.0,       # to keep this notebook concise, we are allowing the pipeline to imput any num of nans
)

In [22]:
classes = pipeline.infer_classes(
    ra=bts_hosts['SN_ra'],
    dec=bts_hosts['SN_dec'],
    redshift=np.array(bts_hosts['sn_redshift'].filled(np.nan)),
    n_resamples=50,
)
print(f'Number of each class:\n{pd.Series(classes).value_counts()}\n\nThe class labels are 0=Ia 1=Ib/c 2=SLSN 3=IIn 4=II (P/L) -1=Outside train properties 4 sigma.')

Associating the catalog!


2025-05-12 12:05:49,383- WARNING - Association failed. Host is likely missing from the catalog.
2025-05-12 12:05:53,864- WARNING - Association failed. Host is likely missing from the catalog.
2025-05-12 12:05:55,954- WARNING - Association failed. Host is likely missing from the catalog.
2025-05-12 12:05:58,207- WARNING - Association failed. Host is likely missing from the catalog.
2025-05-12 12:05:59,823- WARNING - Association failed. Host is likely missing from the catalog.
2025-05-12 12:06:02,378- WARNING - Association failed. Host is likely missing from the catalog.
2025-05-12 12:06:04,523- WARNING - Association failed. Host is likely missing from the catalog.
2025-05-12 12:06:06,273- WARNING - Association failed. Host is likely missing from the catalog.
2025-05-12 12:06:08,222- WARNING - Association failed. Host is likely missing from the catalog.
2025-05-12 12:06:14,755- WARNING - Association failed. Host is likely missing from the catalog.
2025-05-12 12:06:16,901- WARNING - Assoc

Number of each class:
 0    92
-2     4
 4     3
Name: count, dtype: int64

The class labels are 0=Ia 1=Ib/c 2=SLSN 3=IIn 4=II (P/L) -1=Outside train properties 4 sigma.
